In [1]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import numpy as np
import os
import sys
import io

STARTING_GOLD = 500
MAX_HEALTH = 100
MAX_STAMINA = 100
STAT_COLUMNS = ['Health', 'Stamina', 'Damage_Bonus']
BOSS_HEALTH = 150
BOSS_DAMAGE = 15 
BOSS_NAME = "Ogre Boss"

CLEAN_DATA_STRING = """Item_Name,Cost,Weight,Health_Bonus,Stamina_Bonus,Damage_Bonus,Effect_Message
Iron Sword,100,5,0,0,5,"Swings with a dull thud."
Leather Armor,75,8,10,0,0,"Feels light and breathable."
Healing Potion,50,1,-20,0,0,"Restores 20 Health!"
Energy Drink,40,1,0,-5,0,"Restores 5 Stamina!"
Heavy Axe,150,12,0,0,10,"A mighty, slow weapon."
"""


class InventoryManager:
    def __init__(self, start_gold=STARTING_GOLD):
        self.gold = start_gold
        self.items = [] 
    def add_item(self, item_dict):
        self.items.append(item_dict)
    def remove_item(self, item_name):
        for i, item in enumerate(self.items):
            if item['Item_Name'].lower() == item_name.lower():
                removed_item = self.items.pop(i)
                return removed_item
        return None
    
class Character:
    def __init__(self):
        self.base_stats = np.array([MAX_HEALTH, MAX_STAMINA, 0], dtype=np.int32)
        self.loadout_slots = { 
            'Weapon': None, 
            'Armor': None, 
            'Amulet': None 
        } 
    def calculate_current_stats(self, inventory_items):
        current_stats = self.base_stats.copy()
        for item in inventory_items: 
            current_stats[0] += item.get('Health_Bonus', 0) 
            current_stats[1] += item.get('Stamina_Bonus', 0) 
            current_stats[2] += item.get('Damage_Bonus', 0) 
        current_stats[0] = np.clip(current_stats[0], 0, MAX_HEALTH)
        current_stats[1] = np.clip(current_stats[1], 0, MAX_STAMINA)
        return current_stats
    def display_stats(self, inventory_items):
        current_stats = self.calculate_current_stats(inventory_items)
        print("\nCHARACTER STATS")
        for i, stat in enumerate(STAT_COLUMNS):
            print(f"   {stat}: {current_stats[i]}")


def load_shop_data():
    try:
        shop_df = pd.read_csv(
            io.StringIO(CLEAN_DATA_STRING), 
            sep=',', 
            quotechar='"',
            engine='python' 
        )
        expected_cols = ['Item_Name', 'Cost', 'Weight', 'Health_Bonus', 'Stamina_Bonus', 'Damage_Bonus', 'Effect_Message']
        shop_df.columns = expected_cols 
        
        for col in expected_cols[1:-1]:
            shop_df[col] = pd.to_numeric(shop_df[col], errors='coerce').fillna(0).astype(int)
        
        return shop_df.to_dict('records')

    except Exception as e:
        messagebox.showerror("Data Error", f"Failed to load internal shop data: {e}")
        return None

def simulate_combat(char_obj, inv_manager, log_func):
    log_func(f"\nEngaging {BOSS_NAME} (HP: {BOSS_HEALTH})! Prepare for combat!")
    
    current_char_stats = char_obj.calculate_current_stats(inv_manager.items)
    char_health = current_char_stats[0]
    char_damage_bonus = current_char_stats[2]
    
    current_boss_hp = BOSS_HEALTH
    turn = 1
    
    while char_health > 0 and current_boss_hp > 0:
        log_func(f"--- Turn {turn} ---")

        char_attack_power = 10 + char_damage_bonus 
        current_boss_hp -= char_attack_power
        log_func(f"You strike the Ogre, dealing {char_attack_power} damage. Boss HP remaining: {max(0, current_boss_hp)}")

        if current_boss_hp <= 0:
            log_func(f"You defeated the {BOSS_NAME}!")
            inv_manager.gold += 200 
            log_func(f"You gained 200 Gold!")
            break

        boss_damage_taken = BOSS_DAMAGE
        char_health -= boss_damage_taken
        log_func(f"The Ogre hits you for {boss_damage_taken} damage. Your HP remaining: {max(0, char_health)}")
        
        char_obj.base_stats[0] = np.clip(char_health, 0, MAX_HEALTH)

        turn += 1
        
    if char_health <= 0:
        log_func(f"Combat concluded! You were defeated by the {BOSS_NAME}.")
    else:
        log_func(f"You survived the combat!")
        
    return char_health > 0

def get_help_text():
    return ("\nAvailable Commands:\n"
            "COMBAT: fight           (Initiate battle with the Ogre Boss)\n" 
            "CHECK:  check stats     (View current character attributes)\n"
            "        check inventory (View your items)\n"
            "ACTION: shop            (View items available to buy)\n"
            "        buy [Item Name] (Purchase an item)\n"
            "        use [Item Name] (Consume a potion or use gear)\n"
            "SYSTEM: quit            (Exit the game)\n")

def process_command(command, inv_manager, char_obj, shop_items, log_func):
    parts = command.lower().strip().split()
    if not parts: return True
    verb = parts[0]
    
    if verb == 'quit': return False
    
    elif verb == 'check':
        if len(parts) > 1 and parts[1] in ['stats', 'inventory']:
            if parts[1] == 'inventory': log_func("Inventory requested.")
            elif parts[1] == 'stats': log_func("Stats requested.")
        else: log_func("Invalid 'check' command. Use: check inventory or check stats")
    
    elif verb == 'shop':
        shop_output = "\nSHOP ITEMS (Name | Cost | Effect)\n"
        shop_df = pd.DataFrame(shop_items)
        display_cols = ['Item_Name', 'Cost', 'Effect_Message']
        shop_display = shop_df[display_cols].rename(columns={'Item_Name': 'Item', 'Effect_Message': 'Effect'})
        shop_output += shop_display.to_string(index=False)
        log_func(shop_output)
    
    elif verb == 'buy':
        if len(parts) < 2: log_func("Usage: buy [item name]"); return True
        item_name = " ".join(parts[1:])
        item_to_buy = next((item for item in shop_items if item['Item_Name'].lower() == item_name), None)
        if item_to_buy:
            cost = item_to_buy['Cost']
            if inv_manager.gold >= cost:
                inv_manager.gold -= cost
                inv_manager.items.append(item_to_buy) 
                log_func(f"-> Added {item_to_buy['Item_Name']} to inventory.")
            else: log_func(f"Not enough gold to buy {item_to_buy['Item_Name']}. (Cost: {cost})")
        else: log_func(f"Item '{item_name}' not found in shop.")

    elif verb == 'use':
        if len(parts) < 2: log_func("Usage: use [item name]"); return True
        item_name = " ".join(parts[1:])
        used_item = inv_manager.remove_item(item_name)
        if used_item:
            char_obj.base_stats[0] -= used_item.get('Health_Bonus', 0) 
            char_obj.base_stats[1] -= used_item.get('Stamina_Bonus', 0) 
            log_func(f"Used {used_item['Item_Name']}. {used_item['Effect_Message']}")
        else: log_func(f"Item '{item_name}' not found in inventory.")
    
    elif verb == 'fight':
        if char_obj.base_stats[0] <= 0:
            log_func("You are too injured to fight!")
            return True

        success = simulate_combat(char_obj, inv_manager, log_func)
            
    elif command.lower() == 'help':
        log_func(get_help_text())

    else: log_func("Unknown command. Type 'help' to see the list of commands.")

    return True 


class GameGUI:
    def __init__(self, master):
        self.master = master
        master.title("Console RPG Inventory Manager")

        self.shop_items = load_shop_data()
        if not self.shop_items:
            master.destroy()
            return
            
        self.inv_manager = InventoryManager()  
        self.char_obj = Character()

        self.create_widgets()
        self.setup_initial_gear()
        self.update_display()

    def setup_initial_gear(self):
        if self.shop_items:
            initial_item = next((item for item in self.shop_items if item['Item_Name'] == 'Iron Sword'), None)
            if initial_item:
                 self.inv_manager.items.append(initial_item)
        self.log_message("Welcome, Adventurer! Type 'shop' or 'help' below.")

    def create_widgets(self):
        self.log_text = tk.Text(self.master, height=10, width=60, state=tk.DISABLED, bg='light gray')
        self.log_text.grid(row=0, column=0, columnspan=2, padx=10, pady=5)
        
        self.status_label = tk.Label(self.master, justify=tk.LEFT, padx=10, pady=5, text="Status loading...")
        self.status_label.grid(row=1, column=0, columnspan=2, padx=10, pady=5, sticky='w')

        tk.Label(self.master, text="Command:").grid(row=2, column=0, sticky='e')
        self.command_entry = tk.Entry(self.master, width=40)
        self.command_entry.grid(row=2, column=1, padx=10, pady=10, sticky='w')
        self.command_entry.bind('<Return>', self.handle_command_event)
        
        self.send_button = tk.Button(self.master, text="Execute", command=self.handle_command_event)
        self.send_button.grid(row=3, column=1, padx=10, pady=5, sticky='w')

    def log_message(self, message):
        self.log_text.config(state=tk.NORMAL)
        self.log_text.insert(tk.END, message + "\n")
        self.log_text.see(tk.END)
        self.log_text.config(state=tk.DISABLED)

    def handle_command_event(self, event=None):
        command = self.command_entry.get().strip()
        self.command_entry.delete(0, tk.END)
        if not command: return
        self.log_message(f"> {command}")
        process_command(command, self.inv_manager, self.char_obj, self.shop_items, self.log_message)
        self.update_display()
        self.check_game_over()
        
        if command.lower() == 'quit':
            self.master.quit()

    def update_display(self):
        inv_str = f"GOLD: {self.inv_manager.gold}\n"
        if self.inv_manager.items:
            total_weight = sum(item['Weight'] for item in self.inv_manager.items)
            inv_str += f"INVENTORY (Total Weight: {total_weight}):\n"
            item_counts = pd.Series([item['Item_Name'] for item in self.inv_manager.items]).value_counts()
            for name, count in item_counts.items():
                inv_str += f" - {name} (x{count})\n"
        else:
            inv_str += "INVENTORY: Empty\n"

        current_stats = self.char_obj.calculate_current_stats(self.inv_manager.items)
        stat_str = "STATS\n"
        for i, stat in enumerate(STAT_COLUMNS):
            stat_str += f"{stat}: {current_stats[i]}\n"
            
        self.status_label.config(text=f"{inv_str}\n{stat_str}")

    def check_game_over(self):
        current_health = self.char_obj.calculate_current_stats(self.inv_manager.items)[0]
        
        if current_health <= 0 and self.send_button['state'] != tk.DISABLED:
            self.log_message("\nYour health reached zero! Game Over.")
            self.send_button.config(state=tk.DISABLED)

if __name__ == "__main__":
    root = tk.Tk()
    game_app = GameGUI(root)
    root.mainloop()